In [3]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

# Lab 7 Skyscrapers

Recall our exploration of the skyscraper dataset in lecture.
We solved two problems using this dataset: for each city, what's the tallest building for each material?
for each city, what's the height difference between the tallest steel and the tallest concrete building.
I have reviewed our answers below.
We actually had two solutions to the first problem, one using pivot and the other using group.

Today's discussion question: **discuss the differences between 'grouping with 2 categorical variables' and pivot.**

Here is the [**Python Reference**](http://data8.org/sp22/python-reference.html).

In [4]:
sky = Table.read_table('skyscrapers.csv')
sky = (sky.with_column('age', 2024 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

name,material,city,height,age
One World Trade Center,mixed/composite,New York City,541.3,10
Willis Tower,steel,Chicago,442.14,50
432 Park Avenue,concrete,New York City,425.5,9


**For each city, what’s the tallest building for each material?**

In [5]:
# elegant solution using pivot
sky_p = sky.pivot('material', 'city', values='height', collect=max)
sky_p.show(4)

city,concrete,mixed/composite,steel
Atlanta,264.25,311.8,169.47
Austin,208.15,0,93.6
Baltimore,161.24,0,155.15
Boston,121.92,139,240.79


In [6]:
# awkward solution using group
sky.select('material', 'city', 'height').group(['material', 'city'], collect=max).show(1)

material,city,height max
concrete,Atlanta,264.25


**For each city, what’s the height difference between the tallest steel and the tallest concrete building?**

In [7]:
# solution
sky_p.with_column('height diff', sky_p.column(3) - sky_p.column(1)).drop(2)

city,concrete,steel,height diff
Atlanta,264.25,169.47,-94.78
Austin,208.15,93.6,-114.55
Baltimore,161.24,155.15,-6.09001
Boston,121.92,240.79,118.87
Charlotte,265.48,179.23,-86.25
Chicago,423.22,442.14,18.92
Cincinnati,125,175,50
Cleveland,125,215.8,90.8
Columbus,79.25,169.3,90.05
Dallas,176.48,270.06,93.58


I then gave you a takehome problem.\
**Generate a table of the names of the oldest buildings for each 
material for each city.**\
I also gave you some hints.
Please solve that now.

In [8]:
# Hint: You can use sort to find the name of the oldest building in the dataset
# sky.sort('age', descending=True).column('name').item(0)
sky.sort('age', descending=True)

name,material,city,height,age
Statue of Liberty,steel,New York City,93,138
The Rookery,steel,Chicago,55.02,136
Wainwright Building,steel,St. Louis,44.81,132
Park Place Tower,steel,New York City,88.7,129
Reliance Building,steel,Chicago,61.47,129
Bank of Tokyo Building,steel,New York City,103,128
FlatironCity,steel,Atlanta,48.2,127
Ferry Building,steel,San Francisco,74.68,126
Park Row Building,steel,New York City,119.18,125
6 North Michigan,steel,Chicago,85.95,125


In [9]:
'''

# Group by city and material and get the minimum age for each group
age = sky.group(['city', 'material'], min)

# Join back to get the names of the buildings
age = sky.join(['city', 'material', 'age'], sky, ['city', 'material', 'age'])

# Select relevant columns to display
age = sky.select('city', 'material', 'name', 'age')

age.show(5)

'''

age = sky.pivot('material', 'city', values='name', collect=max)
age

city,concrete,mixed/composite,steel
Atlanta,Westin Peachtree Plaza,Two Alliance Center,State of Georgia Building
Austin,Windsor on the Lake,,University of Texas Tower
Baltimore,The John and Frances Angelos Law Center,,Mercy Medical Center Inpatient Tower
Boston,Harbor Towers II,Ellison Building,Two International Place
Charlotte,TradeMark,Hearst Tower,Skye Building
Chicago,theWit Hotel,United Building,Wrigley Building North Addition
Cincinnati,Kroger Building,Great American Tower at Queen City Square,Scripps Center
Cleveland,National City Center,The Metropolitan at the 9,Tower at Erieview
Columbus,Sheraton Columbus Hotel at Capitol Square,,William Green Building
Dallas,W Dallas Victory Hotel & Residences,Comerica Bank Tower,Three AT&T Plaza


## Further exploration

**Find the 10 cities that have the most skyscrapers.**

In [11]:
city_counts = sky.group('city').sort('count', descending=True)

top_10_cities = city_counts.take(np.arange(10))

top_10_cities


city,count
New York City,531
Chicago,358
Miami,86
San Francisco,61
Houston,54
Seattle,54
Honolulu,50
Los Angeles,49
Las Vegas,44
Minneapolis,39


**For each city, find the age of the oldest building in each material.**

In [12]:
oldest_by_material = sky.group(['city', 'material'], min)

oldest_buildings_by_material = oldest_by_material.select('city', 'material', 'age min')

oldest_buildings_by_material.show(5)


city,material,age min
Atlanta,concrete,10
Atlanta,mixed/composite,15
Atlanta,steel,43
Austin,concrete,9
Austin,steel,87


**For each city, find the average height of the buildings for each material.**

In [13]:
average_height_by_material = sky.group(['city', 'material'], np.mean)

average_height_by_material = average_height_by_material.select('city', 'material', 'height mean')

average_height_by_material.show(5)


city,material,height mean
Atlanta,concrete,148.775
Atlanta,mixed/composite,229.24
Atlanta,steel,112.414
Austin,concrete,127.714
Austin,steel,93.6


## Data collection

Build a Birmingham skyscrapers dataset (above 66m), and call it bhmskyscrapers.csv. Use the same csv format as skyscrapers.csv**  The following website is useful: https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Birmingham,_Alabama
This website does not include material.  Find the material of one of the skyscrapers and add it.  

Rerun the above computations with the new dataset that incorporates Birmingham skyscrapers.

Congratulations on finishing lab.

## Bonus challenges

These challenges are optional.

**Bonus: For each city, compute the age difference between oldest concrete and oldest steel building**

In [10]:

skyscrapers = Table.read_table('skyscrapers.csv')

skyscrapers = skyscrapers.with_column('height', skyscrapers.column('height').astype(float))

filtered = skyscrapers.where('material', are.contained_in(['steel', 'concrete']))

oldest_buildings = filtered.group(['city', 'material'], np.min)

steel_buildings = oldest_buildings.where('material', 'steel').select('city', 'completed amin').relabeled('completed amin', 'steel_completed')
concrete_buildings = oldest_buildings.where('material', 'concrete').select('city', 'completed amin').relabeled('completed amin', 'concrete_completed')

joined = steel_buildings.join('city', concrete_buildings)

age_diff = joined.with_column('age_diff', joined.column('steel_completed') - joined.column('concrete_completed'))

age_diff.select('city', 'steel_completed', 'concrete_completed', 'age_diff')

city,steel_completed,concrete_completed,age_diff
Atlanta,1897,1976,-79
Austin,1937,1984,-47
Baltimore,1911,1967,-56
Boston,1915,1971,-56
Charlotte,1924,1992,-68
Chicago,1888,1929,-41
Cincinnati,1913,1954,-41
Cleveland,1924,1980,-56
Columbus,1927,1963,-36
Dallas,1928,1978,-50


**Bonus: What proportion of cities is the first steel earlier?**

In [ ]:

age_diff = joined.with_column('is_steel_earlier', joined.column('steel_completed') < joined.column('concrete_completed'))

steel_earlier_count = age_diff.where('is_steel_earlier', True).num_rows

total_cities = age_diff.num_rows
proportion_steel_earlier = steel_earlier_count / total_cities

proportion_steel_earlier


**Bonus: For each city, find the name of the tallest skyscraper.**

In [ ]:

tallest_buildings = skyscrapers.group('city', max).select('city', 'height max')

tallest_skyscrapers = skyscrapers.join('height', tallest_buildings, 'height max')

tallest_skyscrapers = tallest_skyscrapers.select('city', 'name', 'height')

tallest_skyscrapers